In [19]:
import sys
import os
import math
import pandas as pd
import numpy as np
import json
from math import sqrt
from collections import defaultdict

#import recmetrics

# Python visualisation library
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap

# Another much simpler viz library
import seaborn as sns

# Modelling Libaries
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import mean_squared_error
import sklearn.metrics
from surprise import Reader, Dataset, SVD, evaluate
from collections import defaultdict
from surprise.model_selection import KFold, train_test_split

# Adding root of git repo to path so we can import pyfireanalytics
sys.path.append('..')

# Display formatting options
pd.set_option('display.max_columns', 1000)

In [20]:
data = pd.read_csv('~/desktop/movielens/UDATA.csv', encoding="latin-1")
item = pd.read_csv('~/desktop/movielens/UITEM.csv', encoding="latin-1")
genre = pd.read_csv('~/desktop/movielens/UGENRE.csv', encoding="latin-1")
user = pd.read_csv('~/desktop/movielens/UUSER.csv', encoding="latin-1")
test = pd.read_csv('~/desktop/movielens/U1TEST.csv', encoding="latin-1")
train = pd.read_csv('~/desktop/movielens/U1BASE.csv', encoding="latin-1")

In [21]:
# rename itemid to movieid to join data and item together
data.rename(columns={'itemid':'movieid'}, inplace=True)
test.rename(columns={'itemid':'movieid'}, inplace=True)
train.rename(columns={'itemid':'movieid'}, inplace=True)
#join
df = pd.merge(data, item, on='movieid', how='outer')
df = pd.merge(df, user, on='userid', how='outer')

user2 = pd.merge(user, data, on='userid', how='inner')
user2 = user2.drop('timestamp', 1)
user2 = user2.drop_duplicates()
#change unix timestamp to readable date
df['reviewtimestamp'] = pd.to_datetime(df['timestamp'],unit='s')

#Split reviewtimestamp into year month day
df['reviewyear'] = df['reviewtimestamp'].dt.year
df['reviewmonth'] = df['reviewtimestamp'].dt.month
df['reviewday'] = df['reviewtimestamp'].dt.day

# Model

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df=1, stop_words='english')
tfidf_matrix = tf.fit_transform(item['movie title'])

In [29]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [33]:
titles = item['movie title']
indices = pd.Series(item.index, index=item['movie title'])

def recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]


In [35]:
recommendations('Toy Story (1995)')

1071                    Pyromaniac's Love Story, A (1995)
1052                                  Now and Then (1995)
1546                                     Show, The (1995)
1589                               To Have, or Not (1995)
1343                         Story of Xinghua, The (1993)
477                        Philadelphia Story, The (1940)
547                     NeverEnding Story III, The (1994)
307                        FairyTale: A True Story (1997)
1652    Entertaining Angels: The Dorothy Day Story (1996)
598     Police Story 4: Project S (Chao ji ji hua) (1993)
722                               Boys on the Side (1995)
1146                                     My Family (1995)
938                            Murder in the First (1995)
555                                      Wild Bill (1995)
1519                                     Fear, The (1995)
714                                     To Die For (1995)
1598                        Someone Else's America (1995)
1626          

In [36]:
recommendations('Pinocchio (1940)')

1059                  Adventures of Pinocchio, The (1996)
431                                       Fantasia (1940)
606                                        Rebecca (1940)
477                        Philadelphia Story, The (1940)
493                                His Girl Friday (1940)
523                            Great Dictator, The (1940)
1450                         Foreign Correspondent (1940)
1115                            Mark of Zorro, The (1940)
0                                        Toy Story (1995)
1                                        GoldenEye (1995)
2                                       Four Rooms (1995)
3                                       Get Shorty (1995)
4                                          Copycat (1995)
5       Shanghai Triad (Yao a yao yao dao waipo qiao) ...
6                                   Twelve Monkeys (1995)
7                                             Babe (1995)
8                                 Dead Man Walking (1995)
9             

In [39]:
from surprise import Reader, Dataset, SVD, evaluate

reader = Reader()

data2 = Dataset.load_from_df(data[['userid', 'movieid', 'rating']], reader)

data2.split(n_folds=5)

In [41]:
svd = SVD()

evaluate(svd, data2, measures=['RMSE'])

/Users/kennyliang/anaconda3/lib/python3.6/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/Users/kennyliang/anaconda3/lib/python3.6/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE of algorithm SVD.

------------
Fold 1
RMSE: 0.9399
------------
Fold 2
RMSE: 0.9360
------------
Fold 3
RMSE: 0.9354
------------
Fold 4
RMSE: 0.9358
------------
Fold 5
RMSE: 0.9367
------------
------------
Mean RMSE: 0.9368
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.9399084715086851,
                             0.9360368684309209,
                             0.9354337442130246,
                             0.9358142966432226,
                             0.9366995843160315]})